In [ ]:
import os

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_theme()


def get_best_known_score(instance: str, problem: str):
    file = f"instances/best_scores_{problem}.txt"
    with open(file, "r", encoding="utf8") as file:
        for line in file.readlines():
            instance_, score, optimal = line[:-1].split(" ")
            if instance_ == instance:
                return int(score), optimal == "*"
    raise Exception(f"instance {instance} not found in {file}")


diff_criteria = [
    "random",
    # "deleter",
    "roulette_wheel",
    "pursuit",
    "ucb",
    "neural_net",
]

repertory = "all_methods_all_instances"

problem = "wvcp"

run_time = 3600 * 2

# Choose the set of instances
instances_set = ("pxx", "pxx")
instances_set = ("rxx", "rxx")
instances_set = ("DIMACS_non_optimal", "dimacs_no")
instances_set = ("DIMACS_optimal", "dimacs_o")
instances_set = ("instance_list_wvcp", "all")


with open(f"instances/{instances_set[0]}.txt", "r", encoding="utf8") as f:
    instances = [line.strip() for line in f.readlines()]

title = {
    "iterated": "Iterated",
    "fixed": "Random",
    "random": "Random",
    "deleter": "deleter",
    "roulette_wheel": "Roulette wheel",
    "pursuit": "Pursuit",
    "ucb": "UCB",
    "neural_net": "Neural Net",
    "neural_net_cross": "Neural Net Cross",
    "depth": "depth",
    "depth_fit": "depth_fit",
    "fit": "fit",
    "level": "level",
    "chance": "chance",
}

colors_min_mean_max = sns.color_palette("gray", 3)

# instances = ["R75_5gb"]

for instance in instances:
    # if instance in ("C2000.5","C2000.9", "DSJC125.1gb"):
    #     continue
    # if "DSJC" in instance:
    #     continue
    best_score, optimality = get_best_known_score(instance, problem)
    for rd in range(2):
        fig = plt.figure(tight_layout=True, figsize=(30, 25))
        gs = gridspec.GridSpec(int(len(diff_criteria) / 3) + 1, 3)
        print(f"{instance} - randseed {rd}")

        for n_criteria, criteria in enumerate(diff_criteria):

            file_best_scores = f"outputs/{repertory}/{criteria}/{instance}_{rd}.csv"
            if not os.path.exists(file_best_scores):
                print(f"no file {file_best_scores}")
                continue
            file_tbt = f"outputs/{repertory}/{criteria}/tbt/{instance}_{rd}.csv"
            if not os.path.exists(file_tbt):
                print(f"no file {file_tbt}")
                continue

            # add score over time
            data_score = pd.read_csv(file_best_scores, comment="#")[
                ["score", "turn", "time"]
            ]
            data_score.at[len(data_score) - 1, "time"] = run_time

            with open(file_tbt, "r") as file:
                file.readline()
                operators = [o for o in file.readline()[1:-1].split(":") if o]
                # print(operators)
            # get the data about turn by turn results
            data_tbt = pd.read_csv(file_tbt, comment="#")[
                [
                    "time",
                    "turn",
                    "proba",
                    "selected",
                    "score_pre_ls",
                    "score_post_ls",
                ]
            ]

            # operators = data_tbt.local_search[0].split(":")
            colors = sns.color_palette("husl", len(operators))
            count = {m: [0] for m in operators}
            scatters_x = [[] for _ in range(len(operators) + 1)]
            scatters_y = [[] for _ in range(len(operators) + 1)]
            nb_turn = data_score.turn.iloc[-1] + 1
            current_turn = -1
            for i, (selected, score, score_greedy, turn) in enumerate(
                zip(
                    data_tbt.selected.to_list(),
                    data_tbt.score_post_ls.to_list(),
                    data_tbt.score_pre_ls.to_list(),
                    data_tbt.turn.to_list(),
                )
            ):
                current_turn += 1
                while current_turn != turn:
                    current_turn += 1
                    for nb_o, o in enumerate(operators):
                        if o == "none":
                            pass
                            # count["none"].append(count["none"][-1] + 1)
                        else:
                            count[o].append(count[o][-1])
                for nb_o, o in enumerate(operators):
                    if selected == nb_o:
                        to_add = 1
                        scatters_x[nb_o].append(turn)
                        scatters_y[nb_o].append(score)
                        scatters_x[-1].append(turn)
                        scatters_y[-1].append(score_greedy)
                    else:
                        to_add = 0
                    count[o].append(count[o][-1] + to_add)
            # remove first 0
            for i, o in enumerate(operators):
                count[o].pop(0)

            # create the plot

            ax = fig.add_subplot(gs[n_criteria // 3, n_criteria % 3])
            # for scores
            ax2 = ax.twinx()
            ax2.plot(data_score.turn, data_score.score, label="_score", color="black")

            # print results ls
            # x_space = np.linspace(0, nb_turn, nb_turn)
            # for o in range(len(operators)):
            #     try:
            #         # slope, intercept = np.polyfit(scatters_x[o], scatters_y[o], 1)
            #         # ax2.plot(slope * x_space + intercept, color=colors[o])
            #         ax2.scatter(
            #             scatters_x[o], scatters_y[o], label="_score", color=colors[o]
            #         )
            #     except Exception as e:
            #         print(e, o)
            #         continue
            # if scatters_x[-1]:
            #     slope, intercept = np.polyfit(scatters_x[-1], scatters_y[-1], 1)
            #     ax2.plot(slope * x_space + intercept, color="gray")
            # ax2.scatter(
            #     scatters_x[-1], scatters_y[-1], marker="x", label="greedy", color="gray"
            # )
            ax2.grid(None)
            ax2.set_ylim(best_score if optimality else int(best_score * 0.95))

            # for selection of operators
            for c, o in zip(colors, count.keys()):
                ax.plot(count[o], label=o, color=c)

            plt.xlim(0, nb_turn)
            ax.set_ylim(0)
            ax.set_ylabel("cumulative selections")
            ax.set_xlabel("Iteration")

            color_axis2 = "gray"
            ax.legend(loc="best")
            plt.title(f"{instance} - {title[criteria]}")

        # plt.savefig(f"{instance}_rd_{rand_seed}.png")
        plt.show()


In [ ]:
# 'Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 'cool', 'cool_r', 'coolwarm', 'coolwarm_r', 'copper', 'copper_r', 'crest', 'crest_r', 'cubehelix', 'cubehelix_r', 'flag', 'flag_r', 'flare', 'flare_r', 'gist_earth', 'gist_earth_r', 'gist_gray', 'gist_gray_r', 'gist_heat', 'gist_heat_r', 'gist_ncar', 'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern', 'gist_stern_r', 'gist_yarg', 'gist_yarg_r', 'gnuplot', 'gnuplot2', 'gnuplot2_r', 'gnuplot_r', 'gray', 'gray_r', 'hot', 'hot_r', 'hsv', 'hsv_r', 'icefire', 'icefire_r', 'inferno', 'inferno_r', 'jet', 'jet_r', 'magma', 'magma_r', 'mako', 'mako_r', 'nipy_spectral', 'nipy_spectral_r', 'ocean', 'ocean_r', 'pink', 'pink_r', 'plasma', 'plasma_r', 'prism', 'prism_r', 'rainbow', 'rainbow_r', 'rocket', 'rocket_r', 'seismic', 'seismic_r', 'spring', 'spring_r', 'summer', 'summer_r', 'tab10', 'tab10_r', 'tab20', 'tab20_r', 'tab20b', 'tab20b_r', 'tab20c', 'tab20c_r', 'terrain', 'terrain_r', 'turbo', 'turbo_r', 'twilight', 'twilight_r', 'twilight_shifted', 'twilight_shifted_r', 'viridis', 'viridis_r', 'vlag', 'vlag_r', 'winter', 'winter_r'


In [ ]:
import os

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_theme()


def get_best_known_score(instance: str, problem: str):
    file = f"instances/best_scores_{problem}.txt"
    with open(file, "r", encoding="utf8") as file:
        for line in file.readlines():
            instance_, score, optimal = line[:-1].split(" ")
            if instance_ == instance:
                return int(score), optimal == "*"
    raise Exception(f"instance {instance} not found in {file}")


diff_criteria = [
    # "iterated",
    # "fixed",
    # "random",
    "deleter",
    "roulette_wheel",
    "pursuit",
    "ucb",
    "neural_net",
    # "depth",
    # "depth_fit",
    # "fit",
    # "level",
    # "chance",
]

repertory = "mcts_hh_all"

problem = "wvcp"

run_time = 3600 * 2

# Choose the set of instances
instances_set = ("pxx", "pxx")
instances_set = ("rxx", "rxx")
instances_set = ("DIMACS_non_optimal", "dimacs_no")
instances_set = ("DIMACS_optimal", "dimacs_o")
instances_set = ("../instances_hard_wvcp", "hard_wvcp")
instances_set = ("../instances_non_optimal", "non_optimal")
instances_set = ("instance_list_wvcp", "all")
instances_set = ("../DIMACS_hard", "dimacs_hard")


with open(f"instances/{instances_set[0]}.txt", "r", encoding="utf8") as f:
    instances = [line.strip() for line in f.readlines()]

title = {
    "iterated": "Iterated",
    "fixed": "Random",
    "random": "Random",
    "deleter": "deleter",
    "roulette_wheel": "Roulette wheel",
    "pursuit": "Pursuit",
    "ucb": "UCB",
    "q": "Q",
    "neural_net": "Neural Net",
    "neural_net_cross": "Neural Net Cross",
    "depth": "depth",
    "depth_fit": "depth_fit",
    "fit": "fit",
    "level": "level",
    "chance": "chance",
}


def get_nb_vertices(instance: str):
    file = "instances/instance_info.txt"
    with open(file, "r", encoding="utf8") as file:
        for line in file.readlines():
            instance_, nb_vertices, _ = line[:-1].split(",")
            if instance_ != instance:
                continue
            return int(nb_vertices)
    print(f"instance {instance} not found in instances/instance_info.txt")


operators = ["ILSTS", "RedLS", "TW", "AFISA"]
line_styles = ["-", "--", "-.", ":"]

colors = sns.color_palette("hls", len(operators))

if not os.path.exists("plots"):
    os.mkdir("plots")

for instance in instances:
    fig = plt.figure(tight_layout=True, figsize=(20, 5))
    gs = gridspec.GridSpec(1, 5)
    nb_turns = int(3600 / (0.02 * get_nb_vertices(instance)))

    for n_criteria, criteria in enumerate(diff_criteria):
        # fig, ax = plt.subplots(figsize=(10, 10))
        mean_curve = {o: [] for o in operators}
        for rd in range(20):
            file_tbt = f"outputs/{repertory}/{criteria}/tbt/{instance}_{rd}.csv"
            # get the data about turn by turn results
            data_tbt = pd.read_csv(file_tbt, comment="#")[["selected"]]
            count = {m: [0] for m in operators}
            current_turn = -1
            selected_s = data_tbt.selected.to_list()
            if nb_turns == -1:
                nb_turns = len(selected_s)
            elif nb_turns > len(selected_s):
                print(
                    f"error {instance} {criteria} nb selected got : {len(selected_s)} before : {nb_turns}"
                )
                nb_turns = len(selected_s)
                # print(file_tbt)
                # continue
            # if nb_turns != len(selected_s):
            #     print(instance, criteria, rd, len(selected_s), nb)
            for selected in selected_s:
                for nb_o, o in enumerate(operators):
                    if selected == nb_o:
                        to_add = 1
                    else:
                        to_add = 0
                    count[o].append(count[o][-1] + to_add)
            # remove first 0
            for i, o in enumerate(operators):
                count[o].pop(0)
                mean_curve[o].append(count[o])
        x = np.linspace(0, nb_turns, nb_turns)

        ax = fig.add_subplot(gs[0, n_criteria % 5])
        for i, (o, line_style) in enumerate(zip(operators, line_styles)):
            mean_o = np.array(
                [np.mean([val[i] for val in mean_curve[o]]) for i in range(nb_turns)]
            )
            std_o = np.array(
                [np.std([val[i] for val in mean_curve[o]]) / 2 for i in range(nb_turns)]
            )
            ax.fill_between(
                x,
                mean_o - std_o,
                mean_o + std_o,
                color=colors[i],
                alpha=0.5,
            )
            ax.plot(x, mean_o, line_style, label=o, color=colors[i])

        # handles, labels = plt.gca().get_legend_handles_labels()
        # by_label = dict(zip(labels, handles))
        # plt.legend(
        #     by_label.values(),
        #     by_label.keys(),
        #     bbox_to_anchor=(1.05, 1),
        #     loc="upper left",
        #     borderaxespad=0.0,
        # )
        # plt.grid(True, which="both", linestyle="--")
        # print(nb_turns, instance, criteria)
        ax.set_xlim(0, nb_turns)
        ax.set_ylim(0)
        if n_criteria == 0:
            plt.legend(loc="upper left")
            ax.set_ylabel("cumulative selections")
        ax.set_xlabel("Iterations")

        plt.title(f"{instance} - {title[criteria]}")
        nb_selections = {o:0 for o in operators}
        for o, v in mean_curve.items():
            nb_selections[o] = sum(sum(i) for i in v)
        # sort by nb selections
        nb_selections = {k: v for k, v in sorted(nb_selections.items(), key=lambda item: item[1])}
        print(f"{instance:20} - {title[criteria]:20} - {nb_selections}")
    plt.savefig(f"plot_new/{instance}.png")
    # plt.show()
    plt.close()
